In [1]:
import numpy as np

from utils import GamblerPolicy

In [2]:
p = GamblerPolicy(goal=10, success_probability=0.4)
p.n_states, p.n_nonterminal_states

(11, 9)

In [3]:
p.policy_estimate

array([1, 1, 1, 1, 1, 1, 1, 1, 1])

In [4]:
ptpm = p.define_ptpm()
ptpm

array([[0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ],
       [0.6, 0. , 0.4, 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ],
       [0. , 0.6, 0. , 0.4, 0. , 0. , 0. , 0. , 0. , 0. , 0. ],
       [0. , 0. , 0.6, 0. , 0.4, 0. , 0. , 0. , 0. , 0. , 0. ],
       [0. , 0. , 0. , 0.6, 0. , 0.4, 0. , 0. , 0. , 0. , 0. ],
       [0. , 0. , 0. , 0. , 0.6, 0. , 0.4, 0. , 0. , 0. , 0. ],
       [0. , 0. , 0. , 0. , 0. , 0.6, 0. , 0.4, 0. , 0. , 0. ],
       [0. , 0. , 0. , 0. , 0. , 0. , 0.6, 0. , 0.4, 0. , 0. ],
       [0. , 0. , 0. , 0. , 0. , 0. , 0. , 0.6, 0. , 0.4, 0. ],
       [0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0.6, 0. , 0.4],
       [0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ]])

In [5]:
immediate_rewards = p.immediate_rewards
immediate_rewards

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.])

In [6]:
DISCOUNT = 1

In [9]:
values_of_states = np.zeros(p.n_states)
for i in range(10):
    new_values_of_states = immediate_rewards + DISCOUNT * np.matmul(ptpm, values_of_states)
    new_values_of_states = np.round(new_values_of_states, 4)
    err = (np.abs(values_of_states - new_values_of_states)).sum()
    print(new_values_of_states, err)
    values_of_states = new_values_of_states

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.] 1.0
[0.  0.  0.  0.  0.  0.  0.  0.  0.  0.4 1. ] 0.4
[0.   0.   0.   0.   0.   0.   0.   0.   0.16 0.4  1.  ] 0.16
[0.    0.    0.    0.    0.    0.    0.    0.064 0.16  0.496 1.   ] 0.15999999999999998
[0.     0.     0.     0.     0.     0.     0.0256 0.064  0.2368 0.496
 1.    ] 0.1024
[0.     0.     0.     0.     0.     0.0102 0.0256 0.1101 0.2368 0.5421
 1.    ] 0.10240000000000003
[0.     0.     0.     0.     0.0041 0.0102 0.0502 0.1101 0.2829 0.5421
 1.    ] 0.07479999999999998
[0.     0.     0.     0.0016 0.0041 0.0225 0.0502 0.1433 0.2829 0.5697
 1.    ] 0.07469999999999996
[0.000e+00 0.000e+00 6.000e-04 1.600e-03 1.000e-02 2.250e-02 7.080e-02
 1.433e-01 3.139e-01 5.697e-01 1.000e+00] 0.05810000000000003
[0.000e+00 2.000e-04 6.000e-04 4.400e-03 1.000e-02 3.430e-02 7.080e-02
 1.680e-01 3.139e-01 5.883e-01 1.000e+00] 0.05810000000000006
